In [2]:
"""
Definition
===============================

 items = [monkey, banana, box]

 positions = [0, 1, 2]

OPERATORS
===============================

Move(subject, x1, x2)
PC: monkeyAt(x1), monkeyLevelDown
A: monkeyAt(x2)
D: monkeyAt(x1)

PushBox(x1,x2)
PC: monkeyAt(x1), boxAt(x1), monkeyLevelDown
A: monkeyAt(x2), boxAt(x2)
D: monkeyAt(x1), boxAt(x1)

ClimbBox(x, direction={Up, Down})
PC: monkeyAt(x), boxAt(x), monkeyLevelDown
A: monkeyLevelUp
E: monkeyLevelDown

HaveBanana(x)
PC: monkeyAt(x), bananaAt(x), boxAt(x), monkeyLevelUp
A: GetBananaAt(x)


INITIAL STATE - Properties
===========================

monkeyAt0, monkeyLevelDown, bananaAt1, boxAt2


GOAL STATE - Properties
===========================

GetBanana(at 1)

"""

import pdb
from random import shuffle

class State:
    def __init__(self):
        self.properties = self.generateStates()

    def __eq__(self, other):
        return self.properties == other.properties

    def generateStates(self):
        """
        Generates all possible properties
        :return: list of all properties
        """
        properties = set()

        POSITIONS = [0, 1, 2]
        ELEMENTS = ["monkey", "banana", "box"]

        for el in ELEMENTS:
            for pos in POSITIONS:
                properties.add(el + "At" + str(pos))

        properties.add('monkeyLevelUp')
        properties.add('monkeyLevelDown')
        properties.add('haveBanana')
        return properties

    def setProperties(self, properties):
        self.properties = properties

    def __str__(self):
        return str(self.properties)
    def __repr__(self):
        return str(self.properties)


class Operation:
    def __init__(self, name):
        self.name = name
        self.PC = set()
        self.A = set()
        self.E = set()

    def __str__(self):
        return "{}".format(self.name)

    def __eq__(self, other):
        return self.name == other.name

    def __hash__(self):
        return hash(self.name)

    def canApply(self, state):
        """
        Return T/F if is possible to apply this operation to this state
        :param state: State to test if this Operation could apply here
        :return: True or False based on Preconditions
        """
        return self.PC.intersection(state.properties) == self.PC

    def apply(self, state):
        """
        Return state after apply this operation
        :param state: original state
        :return: new state after apply this operation
        """
        print("Apply {} to state".format(self.name))
        s = State()
        properties = set()
        if self.canApply(state):
            properties = state.properties.union(self.A)
            properties = properties.difference(self.E)
        else:
            print("Cannot apply {} to state {}".format(self.name, state))
        s.setProperties(properties)
        return s

    def show(self):
        """
        Show actions verbose
        :return:
        """
        print(self.name)
        print("PC: {}".format(self.PC))
        print("A: {}".format(self.A))
        print("E: {}".format(self.E))

class Move(Operation):
    def __init__(self, object, x, y):
        self.name = "Move{}({},{})".format(object, x, y)
        self.object = object
        self.x = x
        self.y = y
        self.PC = self.getPreconditions()
        self.A = self.getA()
        self.E = self.getE()

    def getPreconditions(self):
        p = set()
        p.add(self.object + "At" + str(self.x))
        if self.object == 'monkey':
            p.add('monkeyLevelDown')
        return p

    def __repr__(self):
        return Operation.__str__(self)

    def getA(self):
        p = set()
        p.add(self.object + "At" + str(self.y))
        return p

    def getE(self):
        p = set()
        p.add(self.object + "At" + str(self.x))
        return p

class PushBox(Operation):
    def __init__(self, x, y):
        self.name = "PushBox({},{})".format(x, y)
        self.x = x
        self.y = y
        self.PC = self.getPreconditions()
        self.A = self.getA()
        self.E = self.getE()

    def __repr__(self):
        return Operation.__str__(self)

    def getPreconditions(self):
        p = set()
        p.add("boxAt" + str(self.x))
        p.add("monkeyAt" + str(self.x))
        p.add("monkeyLevelDown")
        return p

    def getA(self):
        p = set()
        p.add("boxAt" + str(self.y))
        p.add("monkeyAt" + str(self.y))
        return p

    def getE(self):
        p = set()
        p.add("boxAt" + str(self.x))
        p.add("monkeyAt" + str(self.x))
        return p

class ClimbBox(Operation):
    def __init__(self, x, updown):
        self.name = "ClimbBox{}(at {})".format(updown, x)
        self.x = x
        self.updown = updown
        self.PC = self.getPreconditions()
        self.A = self.getA()
        self.E = self.getE()

    def __repr__(self):
        return Operation.__str__(self)

    def getPreconditions(self):
        p = set()
        p.add("boxAt" + str(self.x))
        p.add("monkeyAt" + str(self.x))
        if self.updown == 'Up':
            p.add("monkeyLevelDown")
        else:
            p.add("monkeyLevelUpAt{}".format(self.x))
        return p

    def getA(self):
        p = set()
        if self.updown == 'Up':
            p.add("monkeyLevelUpAt{}".format(self.x))
        else:
            p.add("monkeyLevelDown".format(self.x))
            p.add("monkeyAt{}".format(self.x))
        return p

    def getE(self):
        p = set()
        if self.updown == 'Up':
            p.add("monkeyLevelDown")
        else:
            p.add("monkeyLevelUpAt{}".format(self.x))
        return p

class HaveBanana(Operation):
    def __init__(self, x):
        self.name = "GetBanana(at {})".format(x)
        self.x = x
        self.PC = self.getPreconditions()
        self.A = self.getA()
        self.E = self.getE()

    def __repr__(self):
        return Operation.__str__(self)

    def getPreconditions(self):
        p = set()
        p.add("boxAt" + str(self.x)) # ? could I ommit this state
        p.add("monkeyAt" + str(self.x))
        p.add("bananaAt" + str(self.x))
        p.add("monkeyLevelUpAt{}".format(self.x))
        return p

    def getA(self):
        p = set()
        p.add("haveBanana")
        return p

    def getE(self):
        p = set()
        return p

def generateOperations(initial):
    """
    Generates all possible operations for some state
    :param initial: State
    :return: list of operations
    """
    operations = list()
    s = State()
    properties = s.generateStates()

    elements = ['banana', 'monkey', 'box']
    movement = ['Up', 'Down']
    positions = [0, 1, 2]
    #Generate Move and PushBox Operations
    for x in positions:
        for y in positions:
            if x!=y:
                operations.append(Move('monkey', x, y))
                operations.append(PushBox(x,y))

        #Generate ClimbBox
        for direction in movement:
            operations.append(ClimbBox(x, direction))

    for item in list(initial.properties):
        if 'banana' in item:
            operations.append(HaveBanana(item.split('At')[1]))
    return operations


def selectOperation(cs, gs):
    """
    Select a list of operations for all possible considering the goal state (gs) as
    intersection of operations A's
    :param cs: initial state
    :param gs: goal state
    :return: list of possible operations
    """
    return list(set(filter(lambda x: len(x.A.intersection(gs)) > 0, generateOperations(cs))))

def isFinalState(state, goal):
    """
    Check if a state is final
    :param state: state to check
    :param goal: goal state
    :return: true or false
    """
    return state.properties.intersection(goal.properties) == goal.properties

def STRIPSiter(state, goal):
    """
    Execute STRIPS (iterative) algorithm to find a sequence of operations
    from initial state to goal state
    :param state: initial state
    :param goal: goal state
    :return: plan of operations or False
    """
    plan = []
    stack = list(goal.properties)
    #Using stack to keep it simple
    while len(stack) > 0:
        #First objective of the stack
        target = stack[0]
        #If objective is action - apply and remove objective. New state is the result
        #of apply this operation
        if type(target) in [ClimbBox, HaveBanana, PushBox, Move]:
            state = target.apply(state)
            plan.append(target)
            stack.remove(target)
        #If target is already in properties, just remove it - Sussman anomaly presents
        #here
        elif target in state.properties:
            stack.remove(target)
        #If target == str means is a property.
        elif type(target) == str:
            #Generate a list of operations which could satisfy this property
            operations = list(selectOperation(state, {target}))
            #pdb.set_trace()
            #If there are operations to consider...
            if len(operations) > 0:
                # Heuristic - if target is about monkey position, filter that operations
                # on Move (PushBox operations also have Add Properties about monkey position, so
                # better to prune)
                if 'monkeyAt' in target:
                    operations = list(filter(lambda x: type(x) == Move, operations))

                # Shuffle operations selected - prevent strange loops
                shuffle(operations)
                # Select one of the randomized operations
                operation = operations[0]
                # Push on top
                stack = [operation] + stack
                # And also push on top the preconditions of the operation
                stack = list(operation.PC) + stack
            #If there are no solution, return false
            else:
                #Break the condition to exit
                stack = []
                plan = False
    return plan

# TESTS
# -----------------------------
# m = Move("monkey", 0, 2)
# pb = PushBox(2,1)
# cb = ClimbBox(1, "Up")
# cb2 = ClimbBox(1, "Down")
# hb = HaveBanana(1)

# s1 = m.apply(initial)
# s2 = pb.apply(s1)
# s3 = cb.apply(s2)

# s4 = cb2.apply(s3)
# s5 = cb.apply(s4)
# s6 = hb.apply(s5)
# print(s6)

initial = State()
#Simple state
#initial.setProperties({"monkeyAt0", "bananaAt1", "boxAt1", "monkeyLevelDown"})

#Another state with Monkey up
#initial.setProperties({"monkeyAt2", "bananaAt1", "boxAt2", "monkeyLevelUpAt2"})

#Lab state
initial.setProperties({"monkeyAt0", "bananaAt1", "boxAt2", "monkeyLevelDown"})

goal = State()
goal.setProperties({'haveBanana'})

print("Initial state: \n---------\n{}\n".format(initial))
print("Goal state: \n---------\n{}\n".format(goal))
print("STRIPS START\n==============")
plan = STRIPSiter(initial, goal)
if plan:
    print("\nDone! Final plan: \n{}".format(plan))
else:
    print("STRIPS did not find solution")

Initial state: 
---------
{'boxAt2', 'bananaAt1', 'monkeyLevelDown', 'monkeyAt0'}

Goal state: 
---------
{'haveBanana'}

STRIPS START
Apply Movemonkey(0,1) to state
Apply Movemonkey(1,2) to state
Apply PushBox(2,1) to state
Apply ClimbBoxUp(at 1) to state
Apply GetBanana(at 1) to state

Done! Final plan: 
[Movemonkey(0,1), Movemonkey(1,2), PushBox(2,1), ClimbBoxUp(at 1), GetBanana(at 1)]


In [1]:
# wumpus world

import random
import numpy as np

class WumpusWorld:
  def __init__(self, blocks, pits, gold, wumpus, initial_location):
    self.initial_location = initial_location    # copy the input
    self.wumpus = wumpus
    self.pits = pits
    self.gold = gold
    self.blocks = blocks
    self.player = self.initial_location
    self.has_arrow = True

    self.breeze = {}    # stores locations of breezy squares
    self.stench = {}    # stores location of smelly squares
    
    for p in self.pits: # initalise breezy squares
      for l in self.neighbours(p):
        self.breeze[l] = True
    for w in self.wumpus: # intialise smelly squares
      for l in self.neighbours(w):
        self.stench[l] = True
      
      
  def neighbours(self, loc):    # returns neighbours of tuple loc = (x,y) 
    return [(loc[0]+1,loc[1]), (loc[0]-1,loc[1]), (loc[0],loc[1]+1), (loc[0],loc[1]-1)]

  def arrow_hits(self, location, dx, dy): # scans to see if the arrow hits
    while location not in self.blocks:
      location = (location[0]+dx, location[1]+dy)
      if location in self.wumpus:
        return True
    return False
  
  def print(self):            # print the board state (useful for debugging)
    print(self.player)
    xmin = min([x for x,y in self.blocks])
    xmax = max([x for x,y in self.blocks])
    ymin = min([y for x,y in self.blocks])
    ymax = max([y for x,y in self.blocks])
    for y in range(ymin, ymax+1):
      for x in range(xmin, xmax+1): 
        
        if (x,ymax-y) in self.blocks:
          print('B',end='')
        elif (x,ymax-y) in self.wumpus:
          print('W',end='')
        elif (x,ymax-y) in self.pits:
          print('P',end='')
        elif (x,ymax-y) in self.gold:
          print('G',end='')
        elif self.player == (x, ymax - y):
          print('Y',end='')
        else:
          print(' ',end='')
      print("")
    b = self.player in self.breeze       # is their square breezy?
    s = self.player in self.stench       # is it smelly?
    print("arrow: " + str(self.has_arrow))
    print("breezy: " + str(b))
    print("stenchy: " + str(s))

    

  def sim(self, agent):
    t = 0
    self.has_arrow = True
    self.player = self.initial_location
    while t < 1000: 
      t+=1

      self.print()

      b = self.player in self.breeze       # is their square breezy?
      s = self.player in self.stench       # is it smelly?
      agent.give_senses(self.player, b, s)  # give the agent its senses
      action = agent.get_action()       # get the agents action
      print(action, end='\n\n')

      new_location = self.player
      if action == 'MOVE_UP':             # update the location for moving up/down/left/right
        new_location = (self.player[0], self.player[1]+1)
      elif action == 'MOVE_DOWN':
        new_location = (self.player[0], self.player[1]-1)
      elif action == 'MOVE_LEFT':
        new_location = (self.player[0]-1,self.player[1])
      elif action == 'MOVE_RIGHT':
        new_location = (self.player[0]+1,self.player[1])
      elif not self.has_arrow and action[0:5] == 'SHOOT':  # check the agent has the arrow if they shot
        return 'NO ARROW'
      elif action == 'SHOOT_UP':                      # check to see if the agent killed the wumpus
        if self.arrow_hits(self.player, 0, 1):
          self.wumpus = {}
          agent.killed_wumpus()
      elif action == 'SHOOT_DOWN':
        if self.arrow_hits(self.player, 0, -1):
          self.wumpus = {}
          agent.killed_wumpus()
      elif action == 'SHOOT_LEFT':
        if self.arrow_hits(self.player, -1, 0):
          self.wumpus = {}
          agent.killed_wumpus()
      elif action == 'SHOOT_RIGHT':
        if self.arrow_hits(self.player, 1, 0):
          self.wumpus = {}
          agent.killed_wumpus()
      elif action == 'QUIT':
        return 'QUIT'

      if action[0:5] == 'SHOOT':      # remove the arrow if it was shot
        self.has_arrow = False

      if new_location in self.pits:   # check if fell into a pit
        return 'FELL'
      if new_location in self.wumpus: # check if eaten by wumpus
        return 'EATEN'
      if new_location in self.gold:   # check if found gold
        return 'GOLD'

      if new_location not in self.blocks: # if agent ran into a wall, then reset position
        self.player = new_location

class Agent:
    def __init__(self):
        self.wump=[['A' for i in range(50)] for j in range(50)]
        self.kb=[['A' for i in range(50)] for j in range(50)]
        self.moves=[]
        self.move=1
        self.tb=False
        self.move_stack=[]
        self.unsafe=[]
        self.border=False
        self.prev=[]
        self.f=False
        self.exp_t=False
        self.shoot=""
        self.arrow_fired=False
        self.visited=[]
        self.a=False
        self.right_border=False
        self.left_border=False
        self.last_move='null'
        self.step_back=False
        self.counter=0
        
    def get_action(self):
        actions = ['MOVE_UP', 'MOVE_DOWN', 'MOVE_LEFT', 'MOVE_RIGHT']
        extras = ['SHOOT_UP', 'SHOOT_DOWN', 'SHOOT_LEFT', 'SHOOT_RIGHT']
        self.counter+=1
        if self.counter>999:
            return "QUIT"
        if self.shoot in extras and self.arrow_fired==False:
            if self.move_p==actions[0]:
                self.last_move=actions[1]
            if self.move_p==actions[1]:
                self.last_move=actions[0]
            if self.move_p==actions[2]:
                self.last_move=actions[3]
            if self.move_p==actions[3]:
                self.last_move=actions[2]
            self.step_back=True
            self.arrow_fired=True
            return self.shoot
        
        if self.step_back==True:
            self.step_back=False
            return self.last_move
            
        if self.f==False:
            if self.exp_t==True:
                #self.unsafe=[
                self.exp_t=False
                self.f=True
            else:
                return(self.explore_world())
        
        if self.f==True:
            t=self.make_move()
            self.f=False
            return t

    def explore_world(self):
        actions = ['MOVE_UP', 'MOVE_DOWN', 'MOVE_LEFT', 'MOVE_RIGHT']
        if self.move==1 and self.tb==False:
            self.tb=True
            self.moves.append(actions[0])
            self.move_p=actions[0]
            return actions[0]
        if self.move==1 and self.tb==True:
            self.tb=False
            self.move=2
            if self.border==False:
                return actions[1]
            else:
                return actions[0]
        
        if self.move==2 and self.tb==False:
            self.tb=True
            self.moves.append(actions[1])
            self.move_p=actions[1]
            return actions[1]
        if self.move==2 and self.tb==True:
            self.tb=False
            self.move=3
            if self.border==False:
                return actions[0]
            else:
                return actions[1]
        
        if self.move==3 and self.tb==False:
            self.tb=True
            self.moves.append(actions[2])
            self.move_p=actions[2]
            return actions[2]
        if self.move==3 and self.tb==True:
            self.tb=False
            self.move=4
            if self.border==False:
                return actions[3]
            else:
                return actions[2]
        
        if self.move==4 and self.tb==False:
            self.tb=True
            self.moves.append(actions[3])
            self.move_p=actions[3]
            return actions[3]
        if self.move==4 and self.tb==True:
            self.tb=False
            self.move=1
            self.moves=[]
            self.exp_t=True
            if self.border==False:
                return actions[2]
            else:
                return actions[3]

    
    def make_move(self):
        self.f=False
        actions = ['MOVE_UP', 'MOVE_DOWN', 'MOVE_LEFT', 'MOVE_RIGHT']
        temp=[]
        t=self.check_pit(self.prev)
        if isinstance(t,int):
            if actions[t] in self.unsafe:
                if len(self.unsafe)!=1:
                    for item in self.unsafe:
                        if item==actions[t]:
                            self.unsafe.remove(item)
        for item in actions:
            if item not in self.unsafe:
                temp.append(item)
        self.unsafe=[]
        self.a=True
        if temp:
            return (random.choice(temp))
        
    def give_senses(self, location, breeze, stench):
        actions = ['MOVE_UP', 'MOVE_DOWN', 'MOVE_LEFT', 'MOVE_RIGHT']
        extras = ['SHOOT_UP', 'SHOOT_DOWN', 'SHOOT_LEFT', 'SHOOT_RIGHT']
        x=location[0]
        y=location[1]
        if stench==True:
            self.wump[x][y]='s'
        if breeze==True:
            self.kb[x][y]='b'
            self.locate_pit(location)
        if breeze==False and stench==False:
            self.kb[x][y]='o'
            self.wump[x][y]='o'            
        if self.prev==location:
            self.border=True
        else:
            self.prev=location
            self.border=False
        if (breeze==True):
            self.unsafe.append(self.move_p)
        if stench==True:
            if self.arrow_fired==False:
                self.unsafe.append(self.move_p)
        c=self.killed_wumpus()
        if c in extras:
            self.shoot=c
        #input()
        #print (np.matrix(self.kb[-1::-1]))
        
    def killed_wumpus(self):
        c=(0,0)
        v=(0,0)
        x=0
        y=0
        l=[]
        l=self.prev
        for i,lst in enumerate(self.wump):
            for j,k in enumerate(lst):
                if k == "s":
                     c=(i, j)
        
        if c:
            x,y=c
        
            if self.wump[x+2][y]=='s':
                self.wump[x+1][y]='w'
            if self.wump[x-2][y]=='s':
                self.wump[x-1][y]='w'
            if self.wump[x+1][y+1]=='s':
                self.wump[x+1][y]='w'
            if self.wump[x+1][y-1]=='s':
                self.wump[x+1][y]='w'
            if self.wump[x-1][y+1]=='s':
                self.wump[x][y+1]='w'
            if self.wump[x-1][y-1]=='s':
                self.wump[x][y-1]='w'

        extras = ['SHOOT_UP', 'SHOOT_DOWN', 'SHOOT_LEFT', 'SHOOT_RIGHT']
        for i,lst in enumerate(self.wump):
            for j,k in enumerate(lst):
                if k == "w":
                     v=(i, j)
                     
        if l[0]==v[0]:
            if l[1]>v[1]:
                return (extras[1])
            else:
                return (extras[0])
        if l[1]==v[1]:
            if l[0]>v[0]:
                return (extras[2])
            else:
                return (extras[3])

    
    def locate_pit(self,location):
        x=location[0]
        y=location[1]
        if self.kb[x+2][y]=='b' and self.kb[x+1][y+1]=='b':
            self.kb[x+1][y]='p'
        if self.kb[x+2][y]=='b' and self.kb[x+1][y-1]=='b':
            self.kb[x+1][y]='p'
        if self.kb[x-2][y]=='b' and self.kb[x-1][y+1]=='b':
            self.kb[x-1][y]='p'
        if self.kb[x-2][y]=='b' and self.kb[x-1][y-1]=='b':
            self.kb[x+1][y]='p'
        if self.kb[x][y+2]=='b' and self.kb[x+1][y+1]=='b':
            self.kb[x][y+1]='p'
        if self.kb[x][y+2]=='b' and self.kb[x-1][y+1]=='b':
            self.kb[x][y+1]='p'
        if self.kb[x][y-2]=='b' and self.kb[x+1][y-1]=='b':
            self.kb[x][y-1]='p'
        if self.kb[x][y-2]=='b' and self.kb[x-1][y-1]=='b':
            self.kb[x][y-1]='p'
        
    
    def check_pit(self,l):
        x=l[0]
        y=l[1]
        if self.kb[x+1][y]=='p':
            return 3
        if self.kb[x-1][y]=='p':
            return 2
        if self.kb[x][y+1]=='p':
            return 0
        if self.kb[x][y-1]=='p':
            return 1

# width of the wumpus playgound
width = 5

# setting the intial parameters
blocks = set()

for x in range(width+1):
    blocks.add((0, x))
    blocks.add((x, 0))
    blocks.add((width,x))
    blocks.add((x, width))

gold = {(4,1)}
pits = {(3,3)}
wumpus_location = {(2,2)}
initial_location = (1,1)

world1 = WumpusWorld(blocks = blocks, gold = gold, wumpus = wumpus_location, pits = pits, initial_location = initial_location)
agent = Agent()
print(world1.sim(agent))


(1, 1)
BBBBBB
B    B
B  P B
B W  B
BY  GB
BBBBBB
arrow: True
breezy: False
stenchy: False
MOVE_UP

(1, 2)
BBBBBB
B    B
B  P B
BYW  B
B   GB
BBBBBB
arrow: True
breezy: False
stenchy: True
MOVE_DOWN

(1, 1)
BBBBBB
B    B
B  P B
B W  B
BY  GB
BBBBBB
arrow: True
breezy: False
stenchy: False
MOVE_DOWN

(1, 1)
BBBBBB
B    B
B  P B
B W  B
BY  GB
BBBBBB
arrow: True
breezy: False
stenchy: False
MOVE_DOWN

(1, 1)
BBBBBB
B    B
B  P B
B W  B
BY  GB
BBBBBB
arrow: True
breezy: False
stenchy: False
MOVE_LEFT

(1, 1)
BBBBBB
B    B
B  P B
B W  B
BY  GB
BBBBBB
arrow: True
breezy: False
stenchy: False
MOVE_LEFT

(1, 1)
BBBBBB
B    B
B  P B
B W  B
BY  GB
BBBBBB
arrow: True
breezy: False
stenchy: False
MOVE_RIGHT

(2, 1)
BBBBBB
B    B
B  P B
B W  B
B Y GB
BBBBBB
arrow: True
breezy: False
stenchy: True
SHOOT_UP

(2, 1)
BBBBBB
B    B
B  P B
B    B
B Y GB
BBBBBB
arrow: False
breezy: False
stenchy: True
MOVE_LEFT

(1, 1)
BBBBBB
B    B
B  P B
B    B
BY  GB
BBBBBB
arrow: False
breezy: False
stenchy: False
MOVE